In [1]:
import os
import struct
import random
import numpy as np
import matplotlib.pyplot as plt

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    #C:/Users/exist/Codes/ai2019/pbl%#2/data/
    #train-patterns-idx3-ubyte
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte'
                                % kind)
    images_path = os.path.join(path,
                               '%s-patterns-idx3-ubyte'
                               % kind)

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII",
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

#### Loading the data

X_train, y_train = load_mnist('./data/',
                              kind='train')
X_test, y_test = load_mnist('./data/',
                            kind='test')

In [2]:
print('train_sample_number:\t:%d, column_number:%d' %(X_train.shape[0], X_train.shape[1]))
print('test_sample_number\t:%d, column_number:%d' %(X_test.shape[0], X_test.shape[1]))

train_sample_number:	:60000, column_number:784
test_sample_number	:10000, column_number:784


In [ ]:
# # SETTING LIMITS OF SETS
LIMIT = 50000
X_train = X_train[:LIMIT]
y_train = y_train[:LIMIT]
X_test = X_test[:LIMIT]
y_test = y_test[:LIMIT]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [3]:
# Preprocessing
X_train=X_train/255
X_test=X_test/255

In [43]:
class myClassifier(object):    
    """
    ovr
    """
    def __init__(self, C=1000, eta=0.001, batch_size=1, epochs=25, epsilon=1e-8, shuffle=True):
        self.C = C
        self.eta = eta
        self.batch_size = batch_size
        self.epochs = epochs
        self.epsilon = epsilon
        self.class_num = 0
        self.shuffle = shuffle
        self.update_count = 0
        
    def fit(self, X, y, params=None, testscore=None):
        X_num, X_fea = np.shape(X)
        #X_num=60000 X_fea=28*28
        self.class_num=len(np.unique(y))
        #class_num=10
        
        if params is None:
            self.params = {
                'aver_w': np.random.randn(self.class_num, X_fea), #(784,10) 정규분포난수
                'aver_b': np.random.randn(self.class_num, 1),
                'w_': np.random.randn(self.class_num, X_fea),
                'b_': np.random.randn(self.class_num, 1)
            }
            w=self.params['w_']
            b=self.params['b_']
        
        if testscore is None:
            self.score_val = 0
        
        cnt=1
        
        for Xi in range(self.epochs):
            #minibatch training
            if self.shuffle:
                s_data, s_labels = self.shuffling(X, y)
                #s_data[60000][28*28]
                #s_labels[60000][1]
                
                encoded_y=self.encoding(s_labels)
                #encoded_y[60000][class_num]
        
            batch_count=X_num/self.batch_size
            
            for t in range(int(batch_count)):
                ###sgd
                batch_X, batch_y, bs=self.batching(s_data, encoded_y, t)
                #batch_X[batch_size][784] batch_y[batch_size][class_num] last_size=?
                
                batch_X=np.reshape(batch_X,(bs,X_fea))
                batch_y=np.reshape(batch_y,(bs,self.class_num))
                
                loss=self.hinge_loss(batch_X, batch_y, w, b)
                #loss[batch_size][class_num]
                loss=1-loss
                
                #loss[loss>=0]=1
                loss[loss<0]=0
                
                l_M_y=loss*batch_y
                #l_M_y  =loss[bs][class_num] batch_y[bs][class_num]
                
                delta_w=np.zeros((self.class_num,X_fea))
                #delta_w[class_num][28*28]
                delta_b=np.zeros((self.class_num,1))
                #delta_b[10][1]
                
                temp_w=np.dot(np.transpose(l_M_y),batch_X)
                #temp_w[10][28*28]=batch_X[batch_size][28*28]
                delta_w=-(1/bs)*temp_w+(1/self.C)*np.array(w)
                #delta_w[10][28*28]=c*temp_w[10][28*28]+c*w[10][28*28]
                
                temp_b=np.sum(np.transpose(l_M_y),axis=1)
                temp_b=np.reshape(temp_b,(self.class_num,1))
                #temp_b[class_num][1]
                delta_b=-(1/bs) * temp_b
                #delta_b[10][1]=c*[class_num][1]
                cnt+=1
                
                ###algorism
                self.update_count += 1
                
                w=w-(self.eta*delta_w)
                #w[class_num][28*28]
                b=np.subtract(b,(self.eta * delta_b))
                #b[class_num][1]
                
            
#             temp2_w=cnt * self.params['aver_w'] + np.array(w)
#             temp2_w=temp2_w/(cnt+1)
#             temp2_b=cnt * self.params['aver_b'] + np.array(b)
#             temp2_b=temp2_b/(cnt+1)
            
            temp2_w=cnt * (cnt/(cnt+1))*self.params['aver_w'] + (1/(cnt+1))*np.array(w)
            temp2_b=cnt * (cnt/(cnt+1))*self.params['aver_b'] + (1/(cnt+1))*np.array(b)
            
            if self.det_weight(X, y, self.params['aver_w'], self.params['aver_b']) < self.det_weight(X, y, temp2_w, temp2_b):
                self.params['aver_w'] = temp2_w
                self.params['aver_b'] = temp2_b
            #self.params['aver_w']=np.where(self.params['aver_w']>temp2_w,self.params['aver_w'],temp2_w)
            #self.params['aver_b']=np.where(self.params['aver_b']>temp2_b,self.params['aver_b'],temp2_b)
            
            prev_score = self.score_val
            pres_score = self.score(X, y)
            print("epoch: ", Xi)
            print("prev_score: %d", prev_score)
            print("pres_score: %d", pres_score)
#             print("aver_w: %d\n", self.params['aver_w'], " w_: %d", self.params['w_'])
#             print("aver_b: %d\n", self.params['aver_b'], " b_: %d", self.params['b_'])
            print('\n')
            if prev_score < pres_score:
                self.score_val = pres_score
                self.params['w_'] = w
                self.params['b_'] = b
        #return w, b
        return self.params['w_'], self.params['b_']#self.params['aver_w'], self.params['aver_b']
        
    def encoding(self, y):
        encoded_y=-1*np.ones((np.shape(y)[0],self.class_num))
        #encoded_y[60000][class_num]
        for i in range(np.shape(y)[0]):
            encoded_y[i,y[i]] = 1
        return encoded_y
                
    def shuffling(self, X, y):
        temp_s=list(zip(X,y))
        random.shuffle(temp_s)
        X,y=zip(*temp_s)
        return X,y
    
    def batching(self, X, y, t):                         
        batch_X=X[t*self.batch_size:min(len(X),(t+1)*self.batch_size)]
        #batch_X[batch_size][28*28]
        batch_y=y[t*self.batch_size:min(len(X),(t+1)*self.batch_size)]
        #batch_y[batch_size][class_num]
        last_size=min(len(X), (t+1)*self.batch_size)-t*self.batch_size
        #last_size[size][28*28]
        
        return batch_X, batch_y,last_size
    
    def hinge_loss(self, X, y, w, b):
        net_v=self.net_input(X,w)
        #net_v[batch_size][class_num]
        temp_l=np.array(net_v)+np.transpose(b)
        #temp_l[batch_size][class_num]
        loss=y*temp_l
        #loss[batch_size][class_num]
        return loss
    
    def net_input(self, X, w):
        #X[batch_size][28*28] w[class_num][28*28]
        net=np.dot(X,np.transpose(w))
        #[batch_size][class_num]
        return net
    
    def test(self,X,y,w,b):
        cor = 0
        err = 0
        net_v=self.net_input(X,w)
        #net_v[batch_size][class_num]
        temp_t=np.add(net_v,np.transpose(b))
        #temp_t[batch_size][class_num]
        p=np.argmax(temp_t,axis=1)
        score = np.mean(p == y)
        
        for i in range(len(X)):
            c = np.argmax(temp_t, axis=1)[i]
            if c == y[i]:
                cor += 1
            else:
                err += 1
        return score, cor, err
    
    def predict(self, X):
        w_=self.params['w_']
        b_=self.params['b_']
        cla_score = self.net_input(w_, X)
        temp=np.add(cla_score,b_)
        temp = temp.T
        pred = np.argmax(temp, axis=1)
        return pred
    
    def score(self, X, y_true):
#         w_ = np.reshape(w_, (10,784))
#         b_ = np.reshape(b_, (10,1))
        pred = self.predict(X)
        score = np.mean(pred == y_true)
        
        return score
    def det_weight(self, X, y, w1, b1):
        cla_score = self.net_input(w1, X)
        temp = np.add(cla_score, b1)
        temp = temp.T
        pred = np.argmax(temp, axis=1)
        sco = np.mean(pred == y)
        
        return sco
    
    def get_params(self, deep=True):
        return {'C':self.C, 'batch_size':self.batch_size, 'epochs':self.epochs}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [44]:
mine=myClassifier()
w,b=mine.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
print(b.shape)
print(w.shape)

In [ ]:
sc = mine.score(X_train, y_train)
print(sc)

In [ ]:
result, cor, err = mine.test(X_train, y_train, w, b)

In [ ]:
print(result)
print(cor)
print(err)

In [ ]:
# GridSearchCV with Customized mode SVM
from sklearn.model_selection import GridSearchCV
params = {
    'C':[0.01, 0.1, 1, 10, 100, 1000],
#     'C':[0.01, 0.1, 1, 10, 100, 1000, 3000]
#     'batch_size':[1, 10, 60, 100, 600]
     'epochs':[5, 10, 25, 50, 200]
}

gs = GridSearchCV(myClassifier(), param_grid=params, cv=3)

In [ ]:
cv = gs.fit(X_train, y_train)

In [ ]:
p = cv.predict(X_train)
print(p)

In [ ]:
sco = cv.score(X_test, y_test)

In [ ]:
print(sco)

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
scoring = {'f1 macro': make_scorer(f1_score , average='macro'), 'f1 micro': make_scorer(f1_score, average = 'micro'), 'Accuracy': make_scorer(accuracy_score) }

svm_grid = GridSearchCV(myClassifier(), params, cv=3, scoring=scoring, refit=False, n_jobs=-1)
svm_grid.fit(X_train, y_train)

svmcv_result = pd.DataFrame(svm_grid.cv_results_)
svmcv_result.to_csv("svmcv_result.csv")